In [1]:
import sys, os
import multiprocessing

logical_cores = os.cpu_count()
physical_cores = multiprocessing.cpu_count()

os.environ["OMP_NUM_THREADS"] = "12"  # set number of OpenMP threads to run in parallel
os.environ["MKL_NUM_THREADS"] = "12"  # set number of MKL threads to run in parallel

import matplotlib.pyplot as plt

from quspin.operators import hamiltonian  # Hamiltonians and operators
from quspin.basis import spin_basis_1d  # Hilbert space spin basis_1d
from quspin.basis.user import user_basis  # Hilbert space user basis
from quspin.basis.user import (
    op_sig_32,
    map_sig_32,
)
from numba import carray, cfunc  # numba helper functions
from numba import uint32, int32  # numba data types

import numpy as np
from scipy.special import comb
np.set_printoptions(precision=24, floatmode='fixed')
from joblib import Parallel, delayed
import time as T
from tqdm import tqdm
from tqdm.notebook import tqdm
import plotly.express as px

def visualize_mat(mat):
    fig = px.imshow(mat, color_continuous_scale='Viridis')
    fig.show()

In [11]:
def ThreeSpinIsing_GS__no_symm_(N, h):
    if N % 3 != 0:
        raise ValueError("System size N must be a multiple of 3.")
        return None
    entire_hilbert_space_basis = spin_basis_1d(N, pauli=True)
    three_spin_list = [[-1.0, i, (i+1) % N, (i+2) % N] for i in range(N)]  # Three-spin interaction
    transverse_field_list = [[-h, i] for i in range(N)]  # Transverse field
    static = [["zzz", three_spin_list], ["x", transverse_field_list]]
    dynamic = []  # No time-dependent terms
    H = hamiltonian(static, dynamic, basis=entire_hilbert_space_basis, dtype=np.float64, check_symm=False, check_herm=False, check_pcon=False)
    E, psi0 = H.eigsh(k=1, which='SA')  # Smallest eigenvalue ('SA' for smallest algebraic)
    norm = np.linalg.norm(psi0)
    if norm != 0:
        psi0 /= norm
    psi0 = np.sign(psi0[0])*psi0
    return E[0], entire_hilbert_space_basis, psi0  # Return ground state energy and eigenvector

def D1_bitmask_pattern__3(N):
    pattern = ['1', '1', '0']
    num_cycles = N // len(pattern)
    remaining_elements = N % len(pattern)
    operator_string = pattern * num_cycles + pattern[:remaining_elements]
    return operator_string
def D2_bitmask_pattern__3(N):
    pattern = ['0', '1', '1']
    num_cycles = N // len(pattern)
    remaining_elements = N % len(pattern)
    operator_string = pattern * num_cycles + pattern[:remaining_elements]
    return operator_string
def D3_bitmask_pattern__3(N):
    pattern = ['1', '0', '1']
    num_cycles = N // len(pattern)
    remaining_elements = N % len(pattern)
    operator_string = pattern * num_cycles + pattern[:remaining_elements]
    return operator_string
# my_sector_blocks = {
# "D_block__": [0, 0, 0],
# "k_block__": 0,
# }

def get_sector_value(sector_name, s_dict):
    return s_dict.get(sector_name, None)

def ThreeSpinIsing_GS__sublattice_symm_3(N, hval_, sector_blocks):
    if N % 3 != 0:
        raise ValueError("System size N must be a multiple of 3.")
        return None
    @cfunc(op_sig_32,locals=dict(s=int32, n=int32, b=uint32))
    def op(op_struct_ptr, op_str, site_ind, N, args):
        op_struct = carray(op_struct_ptr, 1)[0]
        err = 0
        site_ind = N - site_ind - 1                         # convention for QuSpin for mapping from bits to sites.
        n = (op_struct.state >> site_ind) & 1               # either 0 or 1
        s = (((op_struct.state >> site_ind) & 1) << 1) - 1  # either -1 or 1
        b = 1 << site_ind
        if op_str == 120:                                   # "x" is integer value 120 = ord("x")
            op_struct.state ^= b
        elif op_str == 121:                                 # "y" is integer value 120 = ord("y")
            op_struct.state ^= b
            op_struct.matrix_ele *= 1.0j * s
        elif op_str == 43:                                  # "+" is integer value 43 = ord("+")
            if n:
                op_struct.matrix_ele = 0
            else:
                op_struct.state ^= b                        # create spin
        elif op_str == 45:                                  # "-" is integer value 45 = ord("-")
            if n:
                op_struct.state ^= b                        # destroy spin
            else:
                op_struct.matrix_ele = 0
        elif op_str == 122:                                 # "z" is integer value 120 = ord("z")
            op_struct.matrix_ele *= s
        elif op_str == 110:                                 # "n" is integer value 110 = ord("n")
            op_struct.matrix_ele *= n
        elif op_str == 73:                                  # "I" is integer value 73 = ord("I")
            pass
        else:
            op_struct.matrix_ele = 0
            err = -1
        return err
    op_args = np.array([], dtype=np.uint32)
    # ---------------------------------------------------------------
    #######  define symmetry maps  #######
    # ---------------------------------------------------------------
    # the translation operator. In order to be defined with @cfunc decorator it needs to be defined after the variable `N` is assigned in the machine.
    @cfunc(map_sig_32,locals=dict(shift=uint32,xmax=uint32,x1=uint32,x2=uint32,period=int32,l=int32))
    def translation(x, N, sign_ptr, args):
        """works for all system sizes N."""
        shift = args[0]                                     # translate state by shift sites
        period = N                                          # periodicity/cyclicity of translation
        xmax = args[1]
        l = (shift + period) % period
        x1 = x >> (period - l)
        x2 = (x << l) & xmax
        return x2 | x1
    T_args = np.array([1, (1 << N) - 1], dtype=np.uint32)

    # operators using the bitmasks for the D_i symmetries
    @cfunc(map_sig_32)
    def D1(x, N, sign_ptr, args):
        return x ^ args[0]
    D1_args = np.array([int(''.join(D1_bitmask_pattern__3(N)), 2)], dtype=np.uint32)

    @cfunc(map_sig_32)
    def D2(x, N, sign_ptr, args):
        return x ^ args[0]
    D2_args = np.array([int(''.join(D2_bitmask_pattern__3(N)), 2)], dtype=np.uint32)

    @cfunc(map_sig_32)
    def D3(x, N, sign_ptr, args):
        return x ^ args[0]
    D3_args = np.array([int(''.join(D3_bitmask_pattern__3(N)), 2)], dtype=np.uint32)


    D1_quantum_num = get_sector_value("D_block__", sector_blocks)[0]
    D2_quantum_num = get_sector_value("D_block__", sector_blocks)[1]
    D3_quantum_num = get_sector_value("D_block__", sector_blocks)[2]

    momentum_quantum_num = get_sector_value("k_block__", sector_blocks)

    # The dictionary for all the details of the unique sector in which we restrict out choice of bases
    if momentum_quantum_num is None:
        maps = dict(
        # T_block=(translation, N, momentum_quantum_num, T_args),        # tuning off the momentum_sector_fix helps check the depencendies over {D_i}'s alone
        D1_block=(D1, 2, D1_quantum_num, D1_args),
        D2_block=(D2, 2, D2_quantum_num, D2_args),
        D3_block=(D3, 2, D3_quantum_num, D3_args),        
        )       
        op_dict = dict(op=op, op_args=op_args)
        basis = user_basis(
            np.uint32,
            N,
            op_dict,
            allowed_ops=set("+-xyznI"),
            sps=2,
            Ns_block_est = 2**(N-2),         # estimate of number of states in each block when [{D_i} + kblock=0 is] set
            # Ns_block_est=int((2**(N-2))/(N-2)),         # estimate of number of states in each block when [{D_i} + kblock=0 is] set
            **maps,)
    else:
        maps = dict(
        T_block=(translation, N, momentum_quantum_num, T_args),        # tuning off the momentum_sector_fix helps check the depencendies over {D_i}'s alone
        D1_block=(D1, 2, D1_quantum_num, D1_args),
        D2_block=(D2, 2, D2_quantum_num, D2_args),
        D3_block=(D3, 2, D3_quantum_num, D3_args),   
        ) 
        op_dict = dict(op=op, op_args=op_args)
        basis = user_basis(
            np.uint32,
            N,
            op_dict,
            allowed_ops=set("+-xyznI"),
            sps=2,
            Ns_block_est=int((2**(N-2))/(N-1)),         # estimate of number of states in each block when [{D_i} + kblock=0 is] set
            **maps,)    

    
    h=hval_
    three_spin_list = [[-1.0, i, (i+1) % N, (i+2) % N] for i in range(N)]           # Three-spin interaction
    transverse_field_list = [[-h, i] for i in range(N)]                             # Transverse field
    static = [["zzz", three_spin_list], ["x", transverse_field_list]]
    dynamic = []                                                                    # No time-dependent terms
    H = hamiltonian(static, dynamic, basis=basis, dtype=np.float64, check_symm=False, check_herm=False, check_pcon=False)
    gs_energy, psi0 = H.eigsh(k=1, which='SA')  
    norm = np.linalg.norm(psi0)
    if norm != 0:
        psi0 /= norm
    return gs_energy[0], basis, np.sign(psi0[0])*psi0



N = 6
h_VAL = 1.0

st__0 = T.time()


gs_energy, unresrticted_basis, gs_wvfunc = ThreeSpinIsing_GS__no_symm_(N, h_VAL)
print('\nExact diagonalization - No restriction on the Hilbert space')
print(f'Ground state energy of {N} spins = {gs_energy}')
print(f'took {T.time()-st__0:.4f} seconds\n\n')
st__0 = T.time()


my_sector_blocks = {
"D_block__": [0, 0, 0],
"k_block__": 0,
}

print('Exact diagonalization with {D1, D2, D3} symmetries and kblock=0')
gs_energy__, D_restricted_basis, gs_wvfunc__ = ThreeSpinIsing_GS__sublattice_symm_3(N, h_VAL, sector_blocks = my_sector_blocks)
print(f'Ground state energy of {N} spins = {gs_energy__}')
print(f'took {T.time()-st__0:.4f} seconds\n')


Exact diagonalization - No restriction on the Hilbert space
Ground state energy of 6 spins = -7.483314773547887
took 0.0068 seconds


Exact diagonalization with {D1, D2, D3} symmetries and kblock=0
Ground state energy of 6 spins = -7.483314773547885
took 0.4308 seconds



In [46]:
N = 6
h_VAL = 0.0
st__0 = T.time()

print('For this test, only the (D_i) sectors are used without choosing any momentum sector.')
gs_energy, basis, gs_wvfunc = ThreeSpinIsing_GS__no_symm_(N, h_VAL)
print('\nExact diagonalization - No restriction on the Hilbert space')
print(f'Ground state energy of {N} spins = {gs_energy}')
print(f'sector size = {len(basis._basis)}')
print(f'took {T.time()-st__0:.6f} seconds\n\n')
st__0 = T.time()

my_sector_blocks = {"D_block__": [0, 0, 0], "k_block__": None}

print(f'Exact diagonalization with (D1, D2, D3) = (+1, +1, +1) and kblock = {get_sector_value("k_block__", my_sector_blocks)}.')
gs_energy__, D_restricted_basis, gs_wvfunc__ = ThreeSpinIsing_GS__sublattice_symm_3(N, h_VAL, sector_blocks = my_sector_blocks)
print(f'Ground state energy of {N} spins = {gs_energy__}')
print(f'sector size = {len(D_restricted_basis._basis)}')
print(f'took {T.time()-st__0:.6f} seconds\n')

print(f'Error in ground state energy = {abs(gs_energy__ - gs_energy)}')
print(f'With dimensional reduction : (prev_sector_dim / reduced_sector_dim)  = {len(basis._basis)/len(D_restricted_basis._basis)} fold reduction in the Hilbert space\n')


st__0 = T.time()


my_sector_blocks_A = {"D_block__": [1, 0, 1], "k_block__": None}

print(f'Exact diagonalization with (D1, D2, D3) = (-1, +1, -1) and kblock = {get_sector_value("k_block__", my_sector_blocks_A)}.')
gs_energy__A, D_restricted_basis_A, gs_wvfunc__A = ThreeSpinIsing_GS__sublattice_symm_3(N, h_VAL, sector_blocks = my_sector_blocks_A)
print(f'Ground state energy of {N} spins = {gs_energy__A}')
print(f'sector size = {len(D_restricted_basis_A._basis)}')
print(f'took {T.time()-st__0:.6f} seconds\n')
st__0 = T.time()

my_sector_blocks_B = {"D_block__": [1, 1, 0], "k_block__": None}

print(f'Exact diagonalization with (D1, D2, D3) = (-1, -1, +1) and kblock = {get_sector_value("k_block__", my_sector_blocks_B)}.')
gs_energy__B, D_restricted_basis_B, gs_wvfunc__B = ThreeSpinIsing_GS__sublattice_symm_3(N, h_VAL, sector_blocks = my_sector_blocks_B)
print(f'Ground state energy of {N} spins = {gs_energy__B}')
print(f'sector size = {len(D_restricted_basis_B._basis)}')
print(f'took {T.time()-st__0:.6f} seconds\n')
st__0 = T.time()

my_sector_blocks_C = {"D_block__": [0, 1, 1], "k_block__": None}

print(f'Exact diagonalization with (D1, D2, D3) = (+1, -1, -1) and kblock = {get_sector_value("k_block__", my_sector_blocks_C)}.')
gs_energy__C, D_restricted_basis_C, gs_wvfunc__C = ThreeSpinIsing_GS__sublattice_symm_3(N, h_VAL, sector_blocks = my_sector_blocks_C)
print(f'Ground state energy of {N} spins = {gs_energy__C}')
print(f'sector size = {len(D_restricted_basis_C._basis)}')
print(f'took {T.time()-st__0:.6f} seconds\n')
st__0 = T.time()

For this test, only the (D_i) sectors are used without choosing any momentum sector.

Exact diagonalization - No restriction on the Hilbert space
Ground state energy of 6 spins = -6.0
sector size = 64
took 0.003732 seconds


Exact diagonalization with (D1, D2, D3) = (+1, +1, +1) and kblock = None.
Ground state energy of 6 spins = -6.0000000000000036
sector size = 16
took 0.390207 seconds

Error in ground state energy = 3.552713678800501e-15
With dimensional reduction : (prev_sector_dim / reduced_sector_dim)  = 4.0 fold reduction in the Hilbert space

Exact diagonalization with (D1, D2, D3) = (-1, +1, -1) and kblock = None.
Ground state energy of 6 spins = -6.000000000000002
sector size = 16
took 0.305067 seconds

Exact diagonalization with (D1, D2, D3) = (-1, -1, +1) and kblock = None.
Ground state energy of 6 spins = -6.0
sector size = 16
took 0.307015 seconds

Exact diagonalization with (D1, D2, D3) = (+1, -1, -1) and kblock = None.
Ground state energy of 6 spins = -5.999999999999999

In [7]:
vars(basis__)

{'_count_particles': False,
 '_check_herm': None,
 '_check_pcon': None,
 '_check_symm': None,
 '_unique_me': True,
 '_basis_pcon': None,
 '_get_proj_pcon': False,
 '_made_basis': True,
 '_noncommuting_bits': [],
 '_N': 6,
 '_blocks': {'D3_block': 1, 'D2_block': 1, 'D1_block': 1},
 'map_funcs': (<Numba C callback 'ThreeSpinIsing_GS__sublattice_symm_3.<locals>.D3'>,
  <Numba C callback 'ThreeSpinIsing_GS__sublattice_symm_3.<locals>.D2'>,
  <Numba C callback 'ThreeSpinIsing_GS__sublattice_symm_3.<locals>.D1'>),
 '_pers': array([2, 2, 2]),
 '_qs': array([0, 0, 0]),
 'map_args': (array([45], dtype=uint32),
  array([27], dtype=uint32),
  array([54], dtype=uint32)),
 '_basis_dtype': numpy.uint32,
 '_core_args': (64,
  numpy.uint32,
  6,
  2,
  (<Numba C callback 'ThreeSpinIsing_GS__sublattice_symm_3.<locals>.D3'>,
   <Numba C callback 'ThreeSpinIsing_GS__sublattice_symm_3.<locals>.D2'>,
   <Numba C callback 'ThreeSpinIsing_GS__sublattice_symm_3.<locals>.D1'>),
  (2, 2, 2),
  (0, 0, 0),
  (arr

In [4]:
# Assuming `gs_wvfunc` and projection result are NumPy arrays
ordered_sectored_wvfunc = basis__.project_from(gs_wvfunc__).data.flatten()[basis._basis]
ordered_sectored_wvfunc

array([0.390097619653463012490135, 0.390097619653463012490135,
       0.390097619653463012490135, 0.390097619653463012490135,
       0.096439593726307504262252, 0.096439593726307504262252,
       0.096439593726307504262252, 0.096439593726307504262252,
       0.096439593726307504262252, 0.096439593726307504262252,
       0.096439593726307504262252, 0.096439593726307504262252,
       0.096439593726307504262252, 0.096439593726307504262252,
       0.096439593726307504262252, 0.096439593726307504262252,
       0.096439593726307504262252, 0.096439593726307504262252,
       0.096439593726307504262252, 0.096439593726307504262252,
       0.096439593726307504262252, 0.096439593726307504262252,
       0.096439593726307504262252, 0.096439593726307504262252,
       0.096439593726307504262252, 0.096439593726307504262252,
       0.096439593726307504262252, 0.096439593726307504262252,
       0.091456702393964708908136, 0.091456702393964708908136,
       0.091456702393964708908136, 0.091456702393964708

In [213]:
np.inner(gs_wvfunc.flatten(), ordered_sectored_wvfunc)

np.float64(0.6871892206437011)

64

In [ ]:
np.inner(basis.project_from(gs_wvfunc).data, basis__.project_from(gs_wvfunc__).data) 

np.float64(-1.1425125247312903e-16)

In [ ]:
import numpy as np
import plotly.express as px
from scipy.sparse import csr_matrix

def visualize_csr_matrix(mat):
    fig = px.imshow(
        mat.flatten(),
        origin='upper',    # or 'lower' as you prefer
        color_continuous_scale='Viridis',
        labels=dict(color="Value"),
        title="CSR Matrix Visualization"
    )
    fig.update_layout(
        xaxis_title="Columns",
        yaxis_title="Rows"
    )
    fig.show()

visualize_csr_matrix(basis__.get_proj(dtype=np.float64).data)

In [139]:
len(basis__.get_proj(dtype=np.float64).data)

64

In [140]:
len(gs_wvfunc)

64

In [111]:
# vars(basis.project_from(gs_wvfunc))
len(basis.project_from(gs_wvfunc).data)

64

In [116]:
# vars(basis__.project_from(gs_wvfunc__))
len(basis__.project_from(gs_wvfunc__).data)

64

In [146]:
np.linalg.norm(basis.project_from(gs_wvfunc).data) 

np.float64(1.0)

In [147]:
np.linalg.norm(basis__.project_from(gs_wvfunc__).data)

np.float64(1.0)

In [126]:
gs_wvfunc

array([[0.390097619653462790445531],
       [0.096439593726307615284554],
       [0.096439593726307420995525],
       [0.091456702393964570130258],
       [0.096439593726307643040130],
       [0.052880864903441758551050],
       [0.091456702393964722785924],
       [0.057863756235784553905166],
       [0.096439593726307476506676],
       [0.042915082238756306620697],
       [0.052880864903441744673263],
       [0.096439593726307462628888],
       [0.091456702393964695030348],
       [0.096439593726307615284554],
       [0.057863756235784456760651],
       [0.052880864903441751612156],
       [0.096439593726307656917918],
       [0.052880864903441703039899],
       [0.042915082238756202537289],
       [0.096439593726307420995525],
       [0.052880864903441765489944],
       [0.057863756235784435943970],
       [0.096439593726307323851010],
       [0.091456702393964806052651],
       [0.091456702393964570130258],
       [0.096439593726307587528979],
       [0.096439593726307573651191],
 

In [154]:
np.inner(gs_wvfunc.flatten(), basis__.project_from(gs_wvfunc__).data)

np.float64(-1.1005004234861839e-18)

In [117]:
basis.project_from(gs_wvfunc).data - basis__.project_from(gs_wvfunc__).data

array([ 3.372167547500209972000107e-01,  4.355872882286581510014045e-02,
        4.355872882286562081111114e-02,  3.857583749052276994584432e-02,
        3.857583749052313076832732e-02, -4.982891332342753720752171e-03,
        3.359294615818021051412146e-02,  4.163336342344337026588619e-17,
       -1.387778780781445675529540e-17, -5.352451148755118376376672e-02,
       -4.355872882286574571120141e-02, -2.775557561562891351059079e-17,
       -2.775557561562891351059079e-17,  4.982891332342892498630249e-03,
       -3.359294615818026602527269e-02, -3.857583749052297117376753e-02,
        1.942890293094023945741355e-16, -4.355872882286575958898922e-02,
       -5.352451148755126009159966e-02, -4.163336342344337026588619e-17,
        7.632783294297951215412468e-17,  4.982891332342746781858267e-03,
        4.355872882286563468889895e-02,  3.857583749052311689053951e-02,
        4.854162015520826350956085e-02,  5.352451148755128090828137e-02,
        5.352451148755126703049356e-02,  3.47182537

In [96]:
fig = px.imshow(
        basis.get_proj(dtype=np.float64).data,
        origin='upper',    # or 'lower' as you prefer
        color_continuous_scale='Viridis',
        labels=dict(color="Value"),
        title="CSR Matrix Visualization"
    )
fig.show()

ValueError: px.imshow only accepts 2D single-channel, RGB or RGBA images. An image of shape (64,) was provided. Alternatively, 3- or 4-D single or multichannel datasets can be visualized using the `facet_col` or/and `animation_frame` arguments.

In [94]:
vars(basis.get_proj(dtype=np.float64))

{'_shape': (64, 64),
 'maxprint': 50,
 'indices': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63], dtype=int32),
 'indptr': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64],
       dtype=int32),
 'data': array([1.000000000000000000000000, 1.000000000000000000000000,
        1.000000000000000000000000, 1.000000000000000000000000,
        1.000000000000000000000000, 1.000000000000000000000000,
        1.000000000000000000000000, 1.000000000000000000000000,
        1.000000000000000000000000, 1.0000000000000000000000

In [79]:
vars(basis__.get_proj(dtype=np.float64))

{'_shape': (64, 16),
 'maxprint': 50,
 'indices': array([ 0, 27, 45, 54,  1, 26, 44, 55,  2, 25, 47, 52,  3, 24, 46, 53,  4,
        31, 41, 50,  5, 30, 40, 51,  6, 29, 43, 48,  7, 28, 42, 49,  8, 19,
        37, 62,  9, 18, 36, 63, 10, 17, 39, 60, 11, 16, 38, 61, 12, 23, 33,
        58, 13, 22, 32, 59, 14, 21, 35, 56, 15, 20, 34, 57], dtype=int32),
 'indptr': array([ 0,  4,  8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 64],
       dtype=int32),
 'data': array([0.500000000000000000000000, 0.500000000000000000000000,
        0.500000000000000000000000, 0.500000000000000000000000,
        0.500000000000000000000000, 0.500000000000000000000000,
        0.500000000000000000000000, 0.500000000000000000000000,
        0.500000000000000000000000, 0.500000000000000000000000,
        0.500000000000000000000000, 0.500000000000000000000000,
        0.500000000000000000000000, 0.500000000000000000000000,
        0.500000000000000000000000, 0.500000000000000000000000,
        0.5000000000

In [74]:
# vars(basis__.get_proj(dtype=np.float64))
basis__.get_proj(dtype=np.float64).data

visualize_mat(basis__.get_proj(dtype=np.float64).data)

ValueError: px.imshow only accepts 2D single-channel, RGB or RGBA images. An image of shape (64,) was provided. Alternatively, 3- or 4-D single or multichannel datasets can be visualized using the `facet_col` or/and `animation_frame` arguments.

In [54]:

rho_full = gs_wvfunc__ @ gs_wvfunc__.conjugate().T
rho_full /= np.trace(rho_full)  # normalize trace to 1

partial_trace(rho_full, basis, [0], N)

IndexError: index 16 is out of bounds for axis 1 with size 16

In [ ]:

# Build a map from 0..2^N-1 -> (iA, iB)
def get_bits(state_index, sites):
    val = 0
    for s in sites:
        val = (val << 1) | ((state_index >> s) & 1)
    return val



def partial_trace(rho, basis, sub_sys_A, N, sps=2):
    """Trace out the complement of sub_sys_A from an N-qubit density matrix rho."""

    # Sort sub_sys_A for consistency
    sub_sys_A = sorted(sub_sys_A)
    sub_sys_B = [site for site in range(N) if site not in sub_sys_A]

    # Dimensions
    L_A = len(sub_sys_A)
    L_B = N - L_A
    Ns_A = sps**L_A
    Ns_B = sps**L_B

    full_map = []
    for idx in range(sps**N):
        iA = get_bits(idx, sub_sys_A)
        iB = get_bits(idx, sub_sys_B)
        full_map.append((iA, iB))

    # Rearrange into 4D: rho_v[iA, iB, jA, jB]
    rho_v = np.zeros((Ns_A, Ns_B, Ns_A, Ns_B), dtype=rho.dtype)
    relevant_hilbert_space_armlength = len(basis._basis)
    for i in range(relevant_hilbert_space_armlength):
        iA, iB = full_map[i]
        for j in range(relevant_hilbert_space_armlength):
            jA, jB = full_map[j]
            rho_v[iA, iB, jA, jB] = rho[i, j]

    # Partial trace over B => sum over iB = jB
    # Instead of 'iB jA iB -> iA jA', do a valid einsum:
    rho_A = np.einsum('...jlkl->...jk', rho_v)
    # i: iA, j: iB, k: jA, l: jB => summation over j,l

    return rho_A

In [3]:
full__density_matrix_without_restriction = np.outer(gs_wvfunc, gs_wvfunc.conj())
full__density_matrix_with_restriction = np.outer(gs_wvfunc__, gs_wvfunc__.conj())

In [4]:
RDM_size = 2**3
[i for i in range(RDM_size)]

[0, 1, 2, 3, 4, 5, 6, 7]

In [5]:
import scipy

In [6]:
eig_e_s__, eig_v_s__ = scipy.linalg.eigh(full__density_matrix_with_restriction)
eig_e_s__

array([-6.467550949040908605061173e-17, -2.728691996230796812169813e-18,
       -1.993304107891276868714892e-18, -1.595317803738934443311412e-18,
        1.998675198727921669855654e-19,  2.106716282784090948925479e-19,
        2.356868568307116921945112e-19,  5.593498338233368888126345e-19,
        7.843394363373172912103876e-19,  1.156196592181582680704363e-18,
        1.397928407486514567262551e-18,  1.609905024224743067210536e-18,
        2.458536518811410521241132e-18,  3.211179367801860098102363e-18,
        4.440892098500626161694527e-16,  9.999999999999997779553951e-01])

In [7]:
eig_e_s, eig_v_s = scipy.linalg.eigh(unresrticted_basis.ent_entropy(state=gs_wvfunc, sub_sys_A=[1, 2, 3, 4], return_rdm="A")["rdm_A"])

In [8]:
eig_e_s

array([-8.218396549849470271354966e-17, -3.925704952136639281909551e-18,
       -3.660184004075554431467180e-18, -3.453855394816232733568250e-18,
        5.808218017597360673028819e-19,  1.071112553120592669573683e-18,
        1.640898196849078983981166e-18,  2.002316798278275878811483e-18,
        2.860229375838790746171716e-18,  5.706925154211437183118789e-18,
        1.075508955505050766083129e-17,  2.733915173678864458919924e-17,
        9.295712253074300823740828e-02,  9.523809523809524668624960e-02,
        9.523809523809541321970329e-02,  7.165666869930649163222824e-01])

In [9]:
_rdm = full__density_matrix_with_restriction
entropy__ = - np.real(np.trace(_rdm @ scipy.linalg.logm(_rdm)))
entropy__

logm result may be inaccurate, approximate err = 3.2991422044077362e-09


/home/naturalis/miniconda3/envs/QuSpin/lib/python3.10/site-packages/scipy/linalg/_matfuncs.py:200: LogmExactlySingularWarning: The logm input matrix is exactly singular.
  F = scipy.linalg._matfuncs_inv_ssq._logm(A)


np.float64(4.499753539720598e-12)

In [42]:
eigEs, _ = scipy.linalg.eigh(full__density_matrix_without_restriction)
eigEs = eigEs[eigEs > np.finfo(float).eps]
entropy = 0
for i in range(len(eigEs)):
    entropy += -eigEs[i] * np.log(eigEs[i])
entropy

np.float64(1.580980067242042e-14)

In [14]:
eigEs, _ = scipy.linalg.eigh(unresrticted_basis.ent_entropy(state=gs_wvfunc, sub_sys_A=[0, 1, 2, 3], return_rdm="A")["rdm_A"])
eigEs = eigEs[eigEs > np.finfo(float).eps]
entropy = 0
for i in range(len(eigEs)):
    entropy += -eigEs[i] * np.log(eigEs[i])
entropy

np.float64(0.9075317013992835)

In [20]:
eigEs, _ = scipy.linalg.eigh(D_restricted_basis.ent_entropy(state=gs_wvfunc__, sub_sys_A=[0, 1, 2, 3], return_rdm="B")["rdm_B"])
eigEs = eigEs[eigEs > np.finfo(float).eps]
entropy = 0
for i in range(len(eigEs)):
    entropy += -eigEs[i] * np.log(eigEs[i])
entropy

np.float64(0.9075317013992824)

In [33]:
eigEs, _ = scipy.linalg.eigh(unresrticted_basis.ent_entropy(state=gs_wvfunc, sub_sys_A=[3], return_rdm="A")["rdm_A"])
eigEs = eigEs[eigEs > np.finfo(float).eps]
entropy = 0
for i in range(len(eigEs)):
    entropy += -eigEs[i] * np.log(eigEs[i])
entropy

np.float64(0.4835954039610536)

In [74]:
N = 21
h_VAL = 1.0
st__0 = T.time()

print('For this test, only the (D_i) sectors are used without choosing any momentum sector.')
gs_energy, basis, gs_wvfunc = ThreeSpinIsing_GS__no_symm_(N, h_VAL)
print('\nExact diagonalization - No restriction on the Hilbert space')
print(f'Ground state energy of {N} spins = {gs_energy}')
print(f'sector size = {len(basis._basis)}')
print(f'took {T.time()-st__0:.6f} seconds\n')
st__0 = T.time()

print('Exact diagonalization with only {D1, D2, D3} symmetries')
gs_energy__, basis__, gs_wvfunc__ = ThreeSpinIsing_GS__sublattice_symm_3(N, h_VAL)
print(f'Ground state energy of {N} spins = {gs_energy__}')
print(f'sector size = {len(basis__._basis)}')
print(f'took {T.time()-st__0:.6f} seconds\n')

print(f'Error in ground state energy = {abs(gs_energy__-gs_energy)}')
print(f'With dimensional reduction : (prev_sector_dim / reduced_sector_dim)  = {len(basis._basis)/len(basis__._basis)} fold reduction in the Hilbert space')

For this test, only the (D_i) sectors are used without choosing any momentum sector.

Exact diagonalization - No restriction on the Hilbert space
Ground state energy of 21 spins = -25.246267573926932
sector size = 2097152
took 34.760095 seconds

Exact diagonalization with only {D1, D2, D3} symmetries
Ground state energy of 21 spins = -25.246267573926914
sector size = 24980
took 1.415501 seconds

Error in ground state energy = 1.7763568394002505e-14
With dimensional reduction : (prev_sector_dim / reduced_sector_dim)  = 83.95324259407526 fold reduction in the Hilbert space


In [139]:
def RDM(size, h, RDM_size, fix_GS_block = True):
    if RDM_size < size+1:
        if fix_GS_block:
            _, basis_ising, ground_state = ThreeSpinIsing_GS__sublattice_symm_3(size, h)
        else:
            _, basis_ising, ground_state = ThreeSpinIsing_GS__no_symm_(size, h)
        rdm_ = basis_ising.ent_entropy(state=ground_state, sub_sys_A=[i for i in range(RDM_size)], return_rdm="A")["rdm_A"]
        return rdm_
    else:
        raise ValueError('RDM_size > system_size : size of reduced density matrix must be smaller than system size')

def Sigma_h__nn_3(size, h, RDM_size, fix_GS_block, tolerance=1e-9):
    _rdm = RDM(size, h, RDM_size, fix_GS_block)
    rdm__plus = RDM(size, h+tolerance, RDM_size, fix_GS_block)
    log__rdm = scipy.linalg.logm(_rdm)  
    log_rdm__plus = scipy.linalg.logm(rdm__plus)
    d_h__log_rdm = (1/tolerance)*(log_rdm__plus - log__rdm)
    return np.trace(_rdm @ d_h__log_rdm @ d_h__log_rdm).real

RDM(6, 1.0, 1, True)

array([[0.500000000000000333066907, 0.311804782231162147532189],
       [0.311804782231162203043340, 0.500000000000000444089210]])

In [171]:
rdm____ = RDM(18, 0.0, 2, False)

eig_E_s__, eigvs_V___ = scipy.linalg.eigh(rdm____)
fig = px.imshow(rdm____, color_continuous_scale='Viridis')
print(eig_E_s__)
fig.show()

[0.116091164866429785718083 0.133605985058307563662083
 0.306985729685261832244692 0.443317120390000152241328]


In [185]:
rdm____ = RDM(18, 0.0-0.01, 1, True)

eig_E_s__, eigvs_V___ = scipy.linalg.eigh(rdm____)
fig = px.imshow(rdm____, color_continuous_scale='Viridis')
print(eig_E_s__)
fig.show()

[0.498333296294614425114844 0.501666703705386129996668]


In [140]:
RDM(6, 0.0 - 0.0001, 1, fix_GS_block = True)

array([[ 5.000000000000002220446049e-01, -1.666666680566475378100222e-05],
       [-1.666666680563699820538659e-05,  5.000000000000004440892099e-01]])

In [153]:
RDM(6, 0.0, 1, fix_GS_block = False)

array([[ 9.345363126613853710367152e-01, -1.735050382696023335097685e-34],
       [-1.735050382696023335097685e-34,  6.546368733861455957434572e-02]])

In [136]:
fig = px.imshow(RDM(6, 0.0 + 0.0001, 1, fix_GS_block = False), color_continuous_scale='Viridis')
fig.show()

In [102]:
RDM(6, 0.0 - 0.0001, 1, fix_GS_block = False)

array([[ 4.849719672760991473481340e-01, -1.666666677286639123081735e-05],
       [-1.666666677286633702070873e-05,  5.150280327239010746964709e-01]])

In [23]:
import scipy

def entanglement_entropy__1(size, h, RDM_size, fix_GS_block):
    _rdm = RDM(size, h, RDM_size, fix_GS_block)
    return - np.real(np.trace(_rdm @ scipy.linalg.logm(_rdm)))

def entanglement_entropy__2(size, h, RDM_size, fix_GS_block):
    _rdm = RDM(size, h, RDM_size, fix_GS_block)
    eigEs, _ = scipy.linalg.eigh(_rdm)
    eigEs = eigEs[eigEs > np.finfo(float).eps]
    entropy = 0
    for i in range(len(eigEs)):
        entropy += -eigEs[i] * np.log(eigEs[i])
    return entropy

def entanglement_entropy(size, h, RDM_size, fix_GS_block):
    return entanglement_entropy__2(size, h, RDM_size, fix_GS_block)

size = 12
sybsystem_size = 7
h_val = 0.0

print(f'scipy_logm_trace --> {entanglement_entropy__1(size, h_val, sybsystem_size, fix_GS_block=True)}')
print(f'while mine       --> {entanglement_entropy__2(size, h_val, sybsystem_size, fix_GS_block=True)}')

/home/naturalis/miniconda3/envs/QuSpin/lib/python3.10/site-packages/scipy/linalg/_matfuncs.py:200: LogmNearlySingularWarning: The logm input matrix may be nearly singular.
  F = scipy.linalg._matfuncs_inv_ssq._logm(A)


scipy_logm_trace --> 1.38629436111992
while mine       --> 1.3862943611199043


In [30]:
entanglement_entropy(12, 0.0, 10, fix_GS_block = True)

np.float64(1.3862943611198906)

In [35]:
N__ = 12
n__ = 1
h__val = 1.12
entanglement_entropy(N__, h__val, n__, fix_GS_block = True) - entanglement_entropy(N__, h__val, n__, fix_GS_block = True)

np.float64(1.6653345369377348e-15)

In [47]:
Sigma_h__nn_3(12, 0.0, 2, fix_GS_block = True)

np.float64(0.2222214309553445)

In [37]:
Sigma_h__nn_3(12, 0.0, 1, fix_GS_block = False)

np.float64(4.2901054899312403e+17)

In [62]:
Sigma_h__nn_3(3, 1.0, 2, fix_GS_block = True)

np.float64(600.6434824466705)

In [43]:
def generate_susceptibility_data_nn_3(h_values_array, size, rdm_size):
    return np.array(Parallel(n_jobs=12)(
        delayed(Sigma_h__nn_3)(size, h, rdm_size, fix_GS_block=True)
        for h in tqdm(h_values_array, desc=f"SusceptEntEnt: N = {size} || s = {rdm_size}")))

def generate_entanglement_entropy_data_nn_3(h_values_array, size, rdm_size):
    return np.array(Parallel(n_jobs=12)(
        delayed(entanglement_entropy)(size, h, rdm_size, fix_GS_block=True)
        for h in tqdm(h_values_array, desc=f"EntEnt: N = {size} || s = {rdm_size}")))

In [58]:
# h_below = np.linspace(-2, -0.1, 50, endpoint=False)
# h_above = np.linspace(0.1, 2, 50)
# h_values = np.concat([h_below, h_above])
h_values = np.linspace(-2, 2, 500)
len(h_values)

500

In [60]:
_entent_3__1__FB_T = generate_entanglement_entropy_data_nn_3(h_values, 3, 1)
_entent_3__2__FB_T = generate_entanglement_entropy_data_nn_3(h_values, 3, 2)

EntEnt: N = 3 || s = 2:   0%|          | 0/500 [00:00<?, ?it/s]

In [65]:
h_values[250]

np.float64(0.00400801603206391)

In [71]:
Sigma_h__nn_3(3, 0.001 , 2, fix_GS_block=True)

LinAlgError: Singular matrix

In [61]:
_s_data_3__1__FB_T = generate_susceptibility_data_nn_3(h_values, 3, 1)
_s_data_3__2__FB_T = generate_susceptibility_data_nn_3(h_values, 3, 2)

SusceptEntEnt: N = 3 || s = 1:   0%|          | 0/500 [00:00<?, ?it/s]

SusceptEntEnt: N = 3 || s = 2:   0%|          | 0/500 [00:00<?, ?it/s]

LinAlgError: Singular matrix

In [ ]:
plt.figure(figsize=(8*1.2, 5*1.2))

pointSize = 5

plt.scatter(h_values, _entent_3__1__FB_T, label=f'EntEnt: N=3 | rdm_s = 1', s=pointSize)
plt.scatter(h_values, _entent_3__2__FB_T, label=f'EntEnt: N=3 | rdm_s = 2', s=pointSize)


plt.scatter(h_values, _s_data_3__1__FB_T, label=f'Suscept: N=3 | rdm_s = 1', s=pointSize)
plt.scatter(h_values, _s_data_3__2__FB_T, label=f'Suscept: N=3 | rdm_s = 2', s=pointSize)


plt.axvline(x=-1.0000000000, color='black', linestyle='dashed', label="h = - 1.0")
# plt.axvline(x=0.0000000000, color='black', linestyle='solid', label="h = 0.0")
plt.axvline(x=1.0000000000, color='black', linestyle=':', label="h = 1.0")

plt.xlabel('h', fontsize=12)
plt.ylabel('Sigma_h', fontsize=12)
plt.title(f'k_nn_Ising: k=3', fontsize=14)
plt.legend(loc='center', fontsize=10)
plt.show()

In [48]:
N = 6
h_VAL = 1.0
st__0 = T.time()

print('For this test, only the (D_i) sectors are used without choosing any momentum sector.')
gs_energy, basis, gs_wvfunc = ThreeSpinIsing_GS__no_symm_(N, h_VAL)
print('\nExact diagonalization - No restriction on the Hilbert space')
print(f'Ground state energy of {N} spins = {gs_energy}')
print(f'sector size = {len(basis._basis)}')
print(f'took {T.time()-st__0:.6f} seconds\n')
st__0 = T.time()

print('Exact diagonalization with only {D1, D2, D3} symmetries')
gs_energy__, basis__, gs_wvfunc__ = ThreeSpinIsing_GS__sublattice_symm_3(N, h_VAL)
print(f'Ground state energy of {N} spins = {gs_energy__}')
print(f'sector size = {len(basis__._basis)}')
print(f'took {T.time()-st__0:.6f} seconds\n')

print(f'Error in ground state energy = {abs(gs_energy__-gs_energy)}')
print(f'With dimensional reduction : (prev_sector_dim / reduced_sector_dim)  = {len(basis._basis)/len(basis__._basis)} fold reduction in the Hilbert space')

For this test, only the (D_i) sectors are used without choosing any momentum sector.

Exact diagonalization - No restriction on the Hilbert space
Ground state energy of 6 spins = -7.483314773547879
sector size = 64
took 0.008338 seconds

Exact diagonalization with only {D1, D2, D3} symmetries
Ground state energy of 6 spins = -7.483314773547882
sector size = 16
took 0.429908 seconds

Error in ground state energy = 2.6645352591003757e-15
With dimensional reduction : (prev_sector_dim / reduced_sector_dim)  = 4.0 fold reduction in the Hilbert space
